In [1]:
import os
import h2o
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,37 mins 54 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.6
H2O cluster version age:,12 days
H2O cluster name:,HarshithaGS
H2O cluster total nodes:,1
H2O cluster free memory:,3.401 Gb
H2O cluster total cores:,12
H2O cluster allowed cores:,12
H2O cluster status:,"locked, healthy"


In [5]:
## Ingest data
flights_hex = h2o.import_file(path=r'C:\Users\HarshithaGS\Desktop\H2O.ai Demo\flights.csv' ,destination_frame = "flights_hex")
weather_hex = h2o.import_file(path=r'C:\Users\HarshithaGS\Desktop\H2O.ai Demo\weather.csv' ,destination_frame = "weather_hex")

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [6]:
## Summary of the flights and weather dataset
flights_hex.show()
weather_hex.show()

Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,IsArrDelayed,IsDepDelayed
1987,10,3,6,907,907,1511,1500,AA,38,nan,244,233,nan,11,0,SAN,ORD,1723,nan,nan,0,nan,0,0,0,0,0,0,YES,NO
1987,10,6,2,907,907,1507,1500,AA,38,nan,240,233,nan,7,0,SAN,ORD,1723,nan,nan,0,nan,0,0,0,0,0,0,YES,NO
1987,10,8,4,907,907,1451,1500,AA,38,nan,224,233,nan,-9,0,SAN,ORD,1723,nan,nan,0,nan,0,0,0,0,0,0,NO,NO
1987,10,9,5,906,907,1449,1500,AA,38,nan,223,233,nan,-11,-1,SAN,ORD,1723,nan,nan,0,nan,0,0,0,0,0,0,NO,NO
1987,10,10,6,906,907,1434,1500,AA,38,nan,208,233,nan,-26,-1,SAN,ORD,1723,nan,nan,0,nan,0,0,0,0,0,0,NO,NO
1987,10,11,7,907,907,1440,1500,AA,38,nan,213,233,nan,-20,0,SAN,ORD,1723,nan,nan,0,nan,0,0,0,0,0,0,NO,NO
1987,10,12,1,906,907,1456,1500,AA,38,nan,230,233,nan,-4,-1,SAN,ORD,1723,nan,nan,0,nan,0,0,0,0,0,0,NO,NO
1987,10,14,3,906,907,1511,1500,AA,38,nan,245,233,nan,11,-1,SAN,ORD,1723,nan,nan,0,nan,0,0,0,0,0,0,YES,NO
1987,10,15,4,906,907,1444,1500,AA,38,nan,218,233,nan,-16,-1,SAN,ORD,1723,nan,nan,0,nan,0,0,0,0,0,0,NO,NO
1987,10,16,5,907,907,1451,1500,AA,38,nan,224,233,nan,-9,0,SAN,ORD,1723,nan,nan,0,nan,0,0,0,0,0,0,NO,NO


ICAO,MaxTempF,MeanTempF,MinTempF,MaxDewPointF,MeanDewPointF,Min DewpointF,MaxHumidity,MeanHumidity,MinHumidity,MaxSeaLevelPressureIn,MeanSeaLevelPressureIn,MinSeaLevelPressureIn,MaxVisibilityMiles,MeanVisibilityMiles,MinVisibilityMiles,MaxWindSpeedMPH,MeanWindSpeedMPH,MaxGustSpeedMPH,PrecipitationIn,CloudCover,Events,WindDirDegrees,Dest,Year,Month,DayofMonth
KORD,33,32,30,32,29,24,96,90,79,30.14,30.03,29.95,10,3,1,9,5,nan,0.16,8,Fog-Snow,308,ORD,1987,1,1
KORD,36,33,30,33,31,29,100,90,82,30.18,30.02,29.94,8,6,2,10,7,nan,0.01,7,Fog-Snow,323,ORD,1987,1,2
KORD,39,33,27,30,29,26,100,87,65,30.34,30.28,30.19,9,7,5,12,5,nan,0,7,Fog,273,ORD,1987,1,3
KORD,37,28,19,31,26,19,100,89,65,30.42,30.38,30.32,8,4,0,8,4,nan,0,4,Fog,332,ORD,1987,1,4
KORD,39,32,25,30,27,24,100,81,62,30.39,30.27,30.07,10,8,3,13,7,nan,0,5,Fog,180,ORD,1987,1,5
KORD,46,40,34,38,31,26,86,73,62,30.04,29.8,29.61,15,10,10,23,14,nan,0,7,Rain,226,ORD,1987,1,6
KORD,37,35,32,31,28,26,82,75,70,30.27,30.16,29.93,15,11,7,14,10,nan,0,8,,309,ORD,1987,1,7
KORD,32,29,26,27,22,18,85,74,61,30.27,30.22,30.16,15,12,7,17,11,nan,0,7,Snow,268,ORD,1987,1,8
KORD,30,26,21,30,25,19,100,89,82,30.14,29.9,29.61,15,6,0,16,8,nan,0.46,6,Fog-Snow,147,ORD,1987,1,9
KORD,34,32,30,32,29,25,100,89,75,29.91,29.72,29.57,9,4,2,20,12,26,0.15,8,Fog-Snow,339,ORD,1987,1,10


In [7]:
# Group flights by Year
flights_hex["IsArrDelayedNumeric"] = (flights_hex["IsArrDelayed"] == "YES").ifelse(1,0)
flights_hex["IsWeatherDelayedNumeric"] = (flights_hex["WeatherDelay"] > 0).ifelse(1,0)
flights_group = flights_hex.group_by("Year")
flights_count = flights_group.count().sum("IsArrDelayedNumeric").sum("IsWeatherDelayedNumeric").frame
flights_count.as_data_frame()

,Year,nrow,sum_IsArrDelayedNumeric,sum_IsWeatherDelayedNumeric
0,1987,38041,15216,0
1,1988,147295,56686,0
2,1989,133278,59160,0
3,1990,154285,56392,0
4,1991,157913,50490,0
5,1992,163863,57940,0
6,1993,170640,61131,0
7,1994,163451,47078,0
8,1995,150482,44184,0
9,1996,144064,45343,0


In [8]:
## Filter flights before 2003
flights_hex = flights_hex[ flights_hex[ "Year"] >= 2003]
## Filter flights that is delayed but not delayed by weather
flights_hex = flights_hex[ (flights_hex["IsArrDelayed"] == "NO") | (flights_hex["WeatherDelay"] > 0)]

In [9]:
flights_hex["CRSArrTime"]

CRSArrTime
1351
1351
1351
1351
1351
1351
1351
1351
1351
1351


In [10]:
## Parameter Creation
hour1 = flights_hex["CRSArrTime"] // 100
mins1 = flights_hex["CRSArrTime"] % 100
arrTime = hour1*60+mins1
hour2 = flights_hex["CRSDepTime"] // 100
mins2 = flights_hex["CRSDepTime"] % 100
depTime = hour2*60+mins2

travelTime = (arrTime - depTime > 0).ifelse(arrTime - depTime, 0)
flights_hex[ "TravelTime"]  = travelTime

In [11]:
flights_hex[ "TravelTime"]

TravelTime
360
360
360
360
360
360
360
360
360
360


In [12]:
# Set predictor and response variables
myY = "IsArrDelayed"
myX = ["Year", "Month", "DayofMonth", "DayOfWeek", "UniqueCarrier", "Origin", "TravelTime"]

In [26]:
# Create test/train split
split = flights_hex["Year"].runif()
train = flights_hex[split <= 0.75]
valid = flights_hex[split > 0.75]

In [14]:
# GLM - Predict Delays
arr_delay_glm = H2OGeneralizedLinearEstimator(family="binomial",
                                              standardize=True, 
                                              alpha = 0.5, 
                                              lambda_search = True)
arr_delay_glm.train(x               =myX,
                    y               =myY,
                    training_frame  =train,
                    validation_frame=valid)

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [17]:
print ("GLM AUC TRAIN=", arr_delay_glm.auc(train = True),", AUC Valid=",arr_delay_glm.auc(valid = True))

GLM AUC TRAIN= 0.7225639968377122 , AUC Valid= 0.7230479552759056


In [21]:
arr_delay_glm.predict(valid)

glm prediction progress: |████████████████████████████████████████████████| 100%


predict,NO,YES
NO,0.989824,0.0101757
NO,0.989364,0.0106364
NO,0.987221,0.0127786
NO,0.986644,0.0133556
NO,0.986379,0.0136215
NO,0.986107,0.0138926
YES,0.933526,0.0664737
YES,0.932276,0.0677245
YES,0.931884,0.0681164
YES,0.931094,0.0689065


In [27]:
arr_delay_glm.model_performance(valid)


ModelMetricsBinomialGLM: glm
** Reported on test data. **

MSE: 0.020570339942335897
RMSE: 0.14342363801806135
LogLoss: 0.0961101511520657
Null degrees of freedom: 448549
Residual degrees of freedom: 448383
Null deviance: 93406.89078622137
Residual deviance: 86220.41659851815
AIC: 86554.41659851815
AUC: 0.7232113502150174
pr_auc: 0.07329411536481736
Gini: 0.44642270043003474
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.06548295848307542: 


,NO,YES,Error,Rate
NO,429086.0,9785.0,0.0223,(9785.0/438871.0)
YES,8140.0,1539.0,0.841,(8140.0/9679.0)
Total,437226.0,11324.0,0.04,(17925.0/448550.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.0654830,0.1465505,138.0
max f2,0.0337381,0.1944381,229.0
max f0point5,0.1244131,0.1520949,89.0
max accuracy,0.2280680,0.9783770,0.0
max precision,0.2171988,0.2426471,7.0
max recall,0.0023005,1.0,398.0
max specificity,0.2280680,0.9999544,0.0
max absolute_mcc,0.0654830,0.1266267,138.0
max min_per_class_accuracy,0.0203373,0.6574916,289.0
max mean_per_class_accuracy,0.0195012,0.6601671,294.0


Gains/Lift Table: Avg response rate:  2.16 %, avg score:  2.18 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0100011,0.1439095,8.3057174,8.3057174,0.1792243,0.1851908,0.1792243,0.1851908,0.0830664,0.0830664,730.5717408,730.5717408
,2,0.02,0.0697147,5.4040532,6.8550470,0.1166109,0.1124715,0.1479211,0.1488352,0.0540345,0.1371009,440.4053161,585.5047009
,3,0.0300011,0.0631801,3.4090631,5.7063004,0.0735622,0.0657383,0.1231329,0.1211341,0.0340944,0.1711954,240.9063115,470.6300353
,4,0.04,0.0589059,3.1928345,5.0780039,0.0688963,0.0610872,0.1095753,0.1061241,0.0319248,0.2031202,219.2834468,407.8003926
,5,0.0500033,0.0525041,2.5407352,4.5704144,0.0548251,0.0559120,0.0986223,0.0960790,0.0254158,0.2285360,154.0735225,357.0414436
,6,0.1,0.0419469,2.0726668,3.3216241,0.0447249,0.0461836,0.0716754,0.0711330,0.1036264,0.3321624,107.2666778,232.1624135
,7,0.1500011,0.0343322,1.8431245,2.8287836,0.0397717,0.0377268,0.0610407,0.0599974,0.0921583,0.4243207,84.3124526,182.8783607
,8,0.2,0.0295979,1.3431439,2.4573820,0.0289829,0.0318642,0.0530264,0.0529643,0.0671557,0.4914764,34.3143902,145.7381961
,9,0.3,0.0222192,1.1726418,2.0291352,0.0253038,0.0257053,0.0437855,0.0438779,0.1172642,0.6087406,17.2641802,102.9135241
,10,0.4,0.0188758,1.0217998,1.7773014,0.0220488,0.0203357,0.0383514,0.0379924,0.1021800,0.7109205,2.1799773,77.7301374
